In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [18]:
q = """
with playoffs_23_24 as
    (select '2023-24' as season, right(left(product_description,6),2) as round, left(product_description,8) as event_name,
        case
           when round like 'R1' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           when round like 'R2' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           when round like 'R3' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           when round like 'R4' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           else 0
        end as days_out, ticket_type_playoffs, sum(gross_revenue) as gross_revenue, sum(paid_seats) as paid_seats
    from custom.cth_v_ticket_2324_playoffs
    where ticket_type_playoffs in ('Singles', 'Nightly Suites')
        and event_name != '23-24 Pl'
    group by product_description, days_out, ticket_type_playoffs),
playoffs_23_24_total as
    (select '2023-24' as season, right(left(product_description,6),2) as round, left(product_description,8) as event_name,
        case
           when round like 'R1' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           when round like 'R2' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           when round like 'R3' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           when round like 'R4' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           else 0
        end as days_out, 'total' as ticket_type, sum(gross_revenue) as gross_revenue, sum(paid_seats) as paid_seats
    from custom.cth_v_ticket_2324_playoffs
    where ticket_type_playoffs in ('Singles', 'Nightly Suites')
        and event_name != '23-24 Pl'
    group by product_description, days_out),
playoffs_24_25 as
    (select '2023-24' as season, right(left(product_description,6),2) as round, left(product_description,8) as event_name,
        case
           when round like 'R1' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           when round like 'R2' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           when round like 'R3' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           when round like 'R4' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           else 0
        end as days_out, ticket_type_playoffs, sum(gross_revenue) as gross_revenue, sum(paid_seats) as paid_seats
    from custom.cth_v_ticket_2425_playoffs
    where ticket_type_playoffs in ('Singles', 'Nightly Suites')
        and event_name != '24-25 Pl'
    group by product_description, days_out, ticket_type_playoffs),
playoffs_24_25_total as
    (select '2023-24' as season, right(left(product_description,6),2) as round, left(product_description,8) as event_name,
        case
           when round like 'R1' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           when round like 'R2' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           when round like 'R3' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           when round like 'R4' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           else 0
        end as days_out, 'total' as ticket_type, sum(gross_revenue) as gross_revenue, sum(paid_seats) as paid_seats
    from custom.cth_v_ticket_2425_playoffs
    where ticket_type_playoffs in ('Singles', 'Nightly Suites')
        and event_name != '24-25 Pl'
    group by product_description, days_out)
select *
from playoffs_23_24
union all
select *
from playoffs_23_24_total
union all
select *
from playoffs_24_25
union all
select *
from playoffs_24_25_total
order by season, round, event_name, days_out desc
"""
df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [7]:
df

,season,round,event_name,days_out,ticket_type,gross_revenue,paid_seats
0,2023-24,R1,24POR1G1,18,Singles,107225.77,537
1,2023-24,R1,24POR1G1,18,total,132691.77,607
2,2023-24,R1,24POR1G1,18,Nightly Suites,25466.00,70
3,2023-24,R1,24POR1G1,17,Singles,76265.32,438
4,2023-24,R1,24POR1G1,17,Nightly Suites,6955.00,20
...,...,...,...,...,...,...,...
754,2023-24,R4,25POR4G3,19,Nightly Suites,160293.05,80
755,2023-24,R4,25POR4G3,19,Singles,669844.52,548
756,2023-24,R4,25POR4G3,19,total,830137.57,628
757,2023-24,R4,25POR4G3,18,total,97970.34,165


In [8]:
df['min_days_out'] = df.apply(lambda row: df[(df['event_name'] == row['event_name'])&(df['ticket_type'] == row['ticket_type'])]['days_out'].min(), axis = 1)
cumdf = df.groupby(by = ['round','event_name', 'ticket_type'], axis = 0)[['gross_revenue','paid_seats']].cumsum().rename(columns = {'gross_revenue':'cum_gross_rev', 'paid_seats':'cum_num_seats'})
dfdf = pd.concat([df,cumdf], axis = 1)
dfdf['final_seats'] = dfdf.apply(lambda row: dfdf[(dfdf['event_name'] == row['event_name'])&(dfdf['ticket_type'] == row['ticket_type'])&
                                                  (dfdf['days_out'] == row['min_days_out'])]['cum_num_seats'].item(), axis = 1)
dfdf['per_seats_in'] = [x/y for x,y in zip(dfdf['cum_num_seats'],dfdf['final_seats'])]
dfdf['num_seats_left'] = dfdf.apply(lambda row: row['final_seats'] - row['cum_num_seats']+ row['paid_seats'], axis = 1)
dfdf = dfdf[['season','round', 'event_name', 'ticket_type','days_out','gross_revenue','paid_seats','num_seats_left','per_seats_in']]
dfdf

C:\Users\riffere\AppData\Local\Temp\ipykernel_8776\3722283440.py:2: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  cumdf = df.groupby(by = ['round','event_name', 'ticket_type'], axis = 0)[['gross_revenue','paid_seats']].cumsum().rename(columns = {'gross_revenue':'cum_gross_rev', 'paid_seats':'cum_num_seats'})


,season,round,event_name,ticket_type,days_out,gross_revenue,paid_seats,num_seats_left,per_seats_in
0,2023-24,R1,24POR1G1,Singles,18,107225.77,537,2259,0.237716
1,2023-24,R1,24POR1G1,total,18,132691.77,607,2498,0.242994
2,2023-24,R1,24POR1G1,Nightly Suites,18,25466.00,70,239,0.292887
3,2023-24,R1,24POR1G1,Singles,17,76265.32,438,1722,0.431607
4,2023-24,R1,24POR1G1,Nightly Suites,17,6955.00,20,169,0.376569
...,...,...,...,...,...,...,...,...,...
754,2023-24,R4,25POR4G3,Nightly Suites,19,160293.05,80,80,1.000000
755,2023-24,R4,25POR4G3,Singles,19,669844.52,548,713,0.768583
756,2023-24,R4,25POR4G3,total,19,830137.57,628,793,0.791929
757,2023-24,R4,25POR4G3,total,18,97970.34,165,165,1.000000


In [14]:
FLA_Redshift(**rs_creds).write_to_warehouse(df = dfdf, table_name= "cth_historical_tier_velocity_playoffs")

In [15]:
dfr4 = dfdf[dfdf['round']=='R4']

In [16]:
dfr4[dfr4['days_out'] == 10]

,season,round,event_name,ticket_type,days_out,gross_revenue,paid_seats,num_seats_left,per_seats_in,processed_date
660,2023-24,R4,24POR4G3,Singles,10,4922.00,4,901,0.669126,2025-05-30 16:01:09.991854+00:00
661,2023-24,R4,24POR4G3,total,10,4922.00,4,1120,0.655874,2025-05-30 16:01:09.991854+00:00
745,2023-24,R4,25POR4G1,Singles,10,102730.16,244,244,1.000000,2025-05-30 16:01:09.991854+00:00
746,2023-24,R4,25POR4G1,total,10,102730.16,244,244,1.000000,2025-05-30 16:01:09.991854+00:00
